In [23]:
import numpy as np
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine-date')

In [24]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [25]:
from sklearn.model_selection import train_test_split

temp_input, test_input, temp_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [26]:
train_input, val_input, train_target, val_target = train_test_split(temp_input, temp_target, test_size=0.2,
                                                                    random_state=42)

In [27]:
print(train_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [28]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(train_input, train_target)
print(dt.score(train_input, train_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [29]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score

scores = cross_validate(dt, train_input, train_target)
# scores = cross_val_score(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00265503, 0.0060432 , 0.        , 0.        , 0.01563573]), 'score_time': array([0., 0., 0., 0., 0.]), 'test_score': array([0.84975962, 0.85456731, 0.85679904, 0.81588448, 0.84837545])}


In [30]:
print(np.mean(scores['test_score']))

0.8450771776358419


In [32]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.8450771776358419


In [33]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8482153614457832


In [38]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [39]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [40]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [41]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9020928554245851


In [42]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0004}


In [43]:
print(gs.cv_results_['mean_test_score'])

[0.84916892 0.85710624 0.85975395 0.86336261 0.86143693]


In [44]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0004}


In [45]:
params = {
    'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
    'max_depth': range(5, 20, 1),
    'min_samples_split': range(2, 100, 10)
}

In [46]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [47]:
print(gs.best_params_)

{'max_depth': 10, 'min_impurity_decrease': 0.0006000000000000001, 'min_samples_split': 42}


In [48]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8676912663149124


In [51]:
from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10)

array([1, 8, 2, 4, 0, 2, 0, 3, 4, 6], dtype=int64)

In [52]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),
 array([106,  97,  89,  87, 107, 105,  97,  97, 124,  91], dtype=int64))

In [53]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.14828043, 0.13431886, 0.29927575, 0.84599342, 0.45066521,
       0.03222129, 0.24028645, 0.25420425, 0.31177357, 0.17561928])

In [54]:
params = {
    'min_impurity_decrease': uniform(0.0001, 0.001),
    'max_depth': randint(20, 50),
    'min_samples_split': randint(2, 25),
    'min_samples_leaf': randint(1, 25)
}

In [55]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001CCE1BB47D0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001CCE19056D0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001CCE1BB7D90>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001CCE1C46ED0>},
                   random_state=42)

In [56]:
print(gs.best_params_)

{'max_depth': 34, 'min_impurity_decrease': 0.000556069984217036, 'min_samples_leaf': 15, 'min_samples_split': 20}


In [57]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8674514602425253


In [58]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.8607692307692307


In [59]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params, n_iter=100, n_jobs=-1,
                        random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001CCE1BB47D0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001CCE19056D0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001CCE1BB7D90>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001CCE1C46ED0>},
                   random_state=42)

In [61]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.816923076923077
